# 통계기반 자연어 처리 

## NLP의 접근 방식 
1. 규칙 기반 
    - 사람의 규칙(문법 / 패턴)을 정의
2. 통계 기반
    - 단어의 빈도, 확률, 통계를 활용
3. 딥러닝 기반 
    - 대규모 데이터 + 신경망 모델 활용

## N-gram 근사 
- Unigram 
    - N = 1 
    - 독립 단어 (하나의 단어)
- Bigram
    - N = 2
    - 바로 앞의 단어를 고려 
- Trigram
    - N = 3
    - 앞의 2개의 단어를 고려 

## 로그 확률
- 단어의 조건부 확률은 매우 작은 값 (0.0001, 0.00001)
- 조건부의 확률들 끼리 곱하게 되면 -> 0에 가까운 값 -> 계산이 불안정 
- 이러한 문제를 해결하기 위해 log 값을 사용 

## 혼란도(Preplexity)
- 문장을 얼마나 햇갈려하는지를 수치로 표현한 값
- 값이 높다면 -> 문장 이해도가 내려간다.
- 값이 낮다면 -> 문장 이해도가 올라간다. 

### BoW 
- 문서의 단어 순서는 무시 
- 각 문서에서 단어가 몇번 등장했는가?(빈도수)
- 가장 기본적인 백터화 방법 

In [1]:
# 샘플 데이터셋 생성 
docs = [
    "영화가 정말 재미있었다", 
    "영화가 너무 지루하다", 
    "배우의 연기가 너무 좋았다."
]

In [2]:
tokens = [doc.split() for doc in docs]
tokens

[['영화가', '정말', '재미있었다'], ['영화가', '너무', '지루하다'], ['배우의', '연기가', '너무', '좋았다.']]

In [6]:
# tokens에서 각각의 단어들을 하나의 리스트로 생성하고 중복은 제거 
# tokens의 2차원 리스트를 1차원으로 변환 
vocab1 = []
for token in tokens:
    # token -> tokens의 각 원소들 -> 1차원 리스트 
    for word in token:
        # word : token이라는 1차원 리스트의 각각의 원소들 
        vocab1.append(word)
vocab1 = list(set(vocab1))

In [7]:
vocab1

['너무', '지루하다', '배우의', '좋았다.', '연기가', '영화가', '재미있었다', '정말']

In [13]:
vocab = list(set(sum(tokens, [])))

In [24]:
# Bow 행렬을 하나 생성 
# tokens의 문장에 vocab의 단어가 몇개 포함되어있는가?
bow_list = []
vocab.sort()
for doc in tokens:
    # row = [ doc.count(word) for word in vocab ]
    row = []
    for word in vocab:
        # count() 함수는 list에서 인자의 값과 같은 데이터가 몇개 있는가?
        row.append(doc.count(word))
    bow_list.append(row)

In [25]:
import pandas as pd 

In [26]:
df_bow = pd.DataFrame(bow_list, columns=vocab)

In [27]:
df_bow

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다.,지루하다
0,0,0,0,1,1,1,0,0
1,1,0,0,1,0,0,0,1
2,1,1,1,0,0,0,1,0


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

In [30]:
df_vectorizer = pd.DataFrame(X.toarray(), 
    columns = vectorizer.get_feature_names_out())

In [31]:
df_vectorizer

,너무,배우의,연기가,영화가,재미있었다,정말,좋았다,지루하다
0,0,0,0,1,1,1,0,0
1,1,0,0,1,0,0,0,1
2,1,1,1,0,0,0,1,0


### TF-IDF 
- Bow 모델(단순 빈도 모델)의 한계를 보완하기 위한 통계 기반 기법

- TF
    - 문서(문장들의 집합) 내의 빈도가 높을수록 값이 큼
    - 자주 등장 할수록 그 문서에서 중요하다고 판단 
- IDF
    - 문장 내에서의 단어의 희소성

In [32]:
import math 

In [33]:
# 단어사전의 길이 
V = len(vocab)
# 전체 문서의 길이 
N = len(docs)

In [35]:
tokens

[['영화가', '정말', '재미있었다'], ['영화가', '너무', '지루하다'], ['배우의', '연기가', '너무', '좋았다.']]

In [34]:
# 단어의 개수를 생성 
word_cnt = {
    w : sum(1 for doc in tokens if w in doc) for w in vocab
}
word_cnt

{'너무': 2,
 '배우의': 1,
 '연기가': 1,
 '영화가': 2,
 '재미있었다': 1,
 '정말': 1,
 '좋았다.': 1,
 '지루하다': 1}

In [36]:
# TF 계산식 함수 
def tf(word, doc):
    # word : 단어 사전의 각 원소
    # doc : tokens 각 원소 
    # doc.count(word) : 문장에서 특정 단어의 개수 
    # len(doc) : 문장의 단어의 개수
    result = doc.count(word) / len(doc)
    return result

In [37]:
# IDF 계산식 함수 
def idf(word):
    # word : 단어 사전의 각 원소
    # N : docs의 길이 -> 문장들의 개수 
    # word_cnt[word] -> 문서에서 특정 단어의 개수
    result = math.log( (N) / (word_cnt[word] + 1) ) + 1
    return result

In [38]:
# TF-IDF -> TF의 값과 IDF 값을 곱한 수치 
X_tfidf = [ 
    [tf(w, doc) * idf(w) for w in vocab] for doc in tokens 
]
X_tfidf

[[0.0,
  0.0,
  0.0,
  0.3333333333333333,
  0.4684883693693881,
  0.4684883693693881,
  0.0,
  0.0],
 [0.3333333333333333,
  0.0,
  0.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.4684883693693881],
 [0.25,
  0.3513662770270411,
  0.3513662770270411,
  0.0,
  0.0,
  0.0,
  0.3513662770270411,
  0.0]]